In [1]:
import zipfile

with zipfile.ZipFile('Recommendation_systems/hw2/data.zip', 'r') as zip_ref:
    zip_ref.extractall('Recommendation_systems/hw2/temp_data')

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [3]:
data = pd.read_csv('Recommendation_systems/hw2/temp_data/data/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [4]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 0. Товар 999999

#### На вебинаре мы использовали товар 999999 - что это за товар?  
> Это пустой товар который необходим чтобы ввести пользователя(произвести холодный старт). 
#### Зачем он нужен?  
> Необходим чтобы пометить что пользователь не проивзаимодействовал ни с одним товаром
#### Используя этот товар мы смещаем качество рекомендаций. В какую сторону?   
> Думаю что смещение идет в худшую сторону. Почему не знаю, но кажется что этот товар может иметь влияние на сегметирование пользователей и мешать подбору подходящего пользователя
*Можно ли удалить этот товар?
> Думаю да, когда пользователь провзаимодействует с настоящим товаром. Удалить товар и переобучить модель    
#### Уберите этот товар (**внимание**: это можно сделать разными способами!) и сравните с качеством на семинаре.

>т.к. в коде выше он не задан то как вариант убрать 999999
```python
# добавил
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999
# удалил
data_train.loc[data_train['item_id']==999999] = 0
```

- Не вводить его вообще
- Фильтровать выходы (в ручную, ignore_items)
- Разные товары 999999

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. вес = log(sales_sum товара)
- Придумайте пример 3 весов, посчитайте weighted_random_recommendation для разных весов

In [5]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    #! реализовал через клас WeightRandomRecommendation
    #! код разместил только для примера
    recs = np.random.choice(
            items_weights['item_id'], 
            replace=False,
            p=items_weights['weights'], 
            size=n)
    
    return recs.tolist()

In [6]:
%%time

from Recommendation_systems.hw2.baseline import WeightRandomRecommendation
model = WeightRandomRecommendation(
    item_tag='item_id',
    weight_tag='sales_sum'
)
# расчитаем потраченые деньги на каждый товар
data_train['sales_sum'] = data_train['quantity']*data_train['sales_value'] 
model.fit(data_train)
model.recommend()

<timed exec>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Wall time: 242 ms


array([6534178, 6533889, 6534166,  397896, 1426702], dtype=int64)

### Сделайте предсказания

#### Weight Random Recommendation

### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма (с вебинара и weighted_random_recommendation) с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество? Почему?

In [7]:
result = pd.read_csv('Recommendation_systems/hw2/preds.csv')# закгрузка predict с семианара
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[6423444, 1102955, 922898, 14050576, 1978019]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[15778681, 2409151, 9796599, 6533394, 851774]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [8]:
type(result['actual'][0])

str

In [10]:
# файл с предсказаниями сохранил предсказания как строки
# нужно перевести обратно в список
for a in result.columns[1:]:
    result.loc[:, a]=  result[a].map(lambda x: x[1:-1].split(', ')).apply(lambda x: list(map(int, x)))

In [11]:
type(result['own_purchases'][0])

list

In [12]:
result['weight_random_recommendation'] = result['user_id'].apply(lambda user_id: model.recommend())
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,weight_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[6423444, 1102955, 922898, 14050576, 1978019]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]","[6533889, 6534178, 6534166, 480014, 6544236]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[15778681, 2409151, 9796599, 6533394, 851774]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]","[6544236, 6534178, 6533889, 6534166, 1404121]"


In [13]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [14]:
result.apply(lambda x: precision_at_k(x['own_purchases'], x['actual'],  5), axis=1).mean()

0.17969311132876264

In [15]:
from Recommendation_systems.hw2.metric import MetricManager

metrics = MetricManager()
for index in result.index:
    item = result.iloc[index]
    if len(item['random_recommendation'])==5:           # на некторых позициях длинна списка меньше 5
        metrics.apply(        
            recommended = item['random_recommendation'],
            boughted = item['actual'],
            name = 'random_recommendation',
            user = item['user_id']
            )
    if len(item['popular_recommendation'])==5:          # на некторых позициях длинна списка меньше 5
        metrics.apply(        
            recommended = item['popular_recommendation'],
            boughted = item['actual'],
            name = 'popular_recommendation',
            user = item['user_id']
            )
    if len(item['itemitem'])==5:                        # на некторых позициях длинна списка меньше 5
        metrics.apply(        
            recommended = item['itemitem'],
            boughted = item['actual'],
            name = 'itemitem',
            user = item['user_id']
            )
    if len(item['cosine'])==5:                          # на некторых позициях длинна списка меньше 5
        metrics.apply(        
            recommended = item['cosine'],
            boughted = item['actual'],
            name = 'cosine',
            user = item['user_id']
            )
    if len(item['tfidf'])==5:                           # на некторых позициях длинна списка меньше 5
        metrics.apply(        
            recommended = item['tfidf'],
            boughted = item['actual'],
            name = 'tfidf',
            user = item['user_id']
            )
    if len(item['own_purchases'])==5:                   # на некторых позициях длинна списка меньше 5
        metrics.apply(        
            recommended = item['own_purchases'],
            boughted = item['actual'],
            name = 'own_purchases',
            user = item['user_id']
            )
    if len(item['weight_random_recommendation'])==5:    # на некторых позициях длинна списка меньше 5 
        metrics.apply(        
            recommended = item['weight_random_recommendation'],
            boughted = item['actual'],
            name = 'weight_random_recommendation',
            user = item['user_id']
            )
metrics.show_table()


Name model                      Hit rate    Precision    Average precision    DCG    NDCG    MRR
----------------------------  ----------  -----------  -------------------  -----  ------  -----
random_recommendation              0.003        0.001                0      0.001   0.001  0
popular_recommendation             0.531        0.155                0.083  0.202   0.174  0.069
itemitem                           0.487        0.137                0.065  0.137   0.118  0.053
cosine                             0.476        0.133                0.105  0.142   0.123  0.091
tfidf                              0.485        0.139                0.111  0.146   0.126  0.094
own_purchases                      0.581        0.181                0.086  0.194   0.168  0.066
weight_random_recommendation       0.22         0.046                0.039  0.046   0.039  0.038


### weight_random_recommendation 
* Только в ~22% был хотя бы один релевантный товар
* Только ~5% проценов куплено товаров из рекомендованых

В некоторых рекомендациях коллекции длинной меньше 5 и чтобы Average precision был расчитан все что меньше 5 не брал в ращёт. С этими рекомендациями надо разбираться

### Задание 3. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.
- Попробуйте стратегии ансамблирования изученных алгоритмов


In [16]:
# your_code

### Задание 4*. Улучшение детерминированных алгоритмов
На семинаре мы рассматривали 



Далее $U \equiv N_i(u) $

$$r_{u,i} =  \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)r_{v, i}$$
$$ S = \sum\limits_{v \in U} \operatorname{sim}(u,v)$$

Предлагается улучшить эту формулу и учесть средние предпочтения всех пользователей

$$r_{u,i} = \mu + \bar{r_u} + \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)(r_{v, i}-\bar{r_{v}} - \mu)$$

Какие смысл имееют $ \mu $ и $ \bar{r_u}$ ?

Реализуйте алгоритм, прогнозирующий рейтинги на основе данной формулы, на numpy (векторизованно!)

В качестве схожести возьмите CosineSimilarity.

Примените к user_item_matrix. В качестве рейтингов возьмите количество или стоимость купленного товара. 
Данный алгоритм предсказывает рейтинги. Как на основании предсказанных рейтингов предсказать факт покупки?

Предложите вариант.
Посчитайте accuracy@5 и сравните с алгоритмами, разобранными на вебинаре.